In [1]:
#Copy and collect the right pictures from the source:
import pandas as pd
import numpy as np

missing_anchors = False
missing_unlabeled = False

In [2]:
# Paths to full-collection of pictures
# (Jonathan's pc):
path_2017 = r"C:\Users\COBOD\3D Objects\ISIC-2017_Training_Data"
path_2020 = r"C:\Users\COBOD\3D Objects\ISIC_2020_Training_Dicom.zip"

In [3]:
# Path to destination of collections:
#C:\Users\COBOD\OneDrive\CBS doku\ITU\3semester\Project\Data
dest_2017 = "anchor_images"
dest_2020 = "unlabeled_images"

In [4]:
# Create list of anchor images needed:
df_anchor = pd.read_csv("ISIC-2017_anchor_images.csv")
images_2017 = df_anchor["image_id"].values.tolist()
#print(images_2017)

In [5]:
# Create list of unlabeled images needed:
df_unlabeled = pd.read_csv("ISIC_2020_unlabeled_images.csv")
images_2020 = df_unlabeled["image_id"].values.tolist()
#print(images_2020)

In [6]:
import shutil, zipfile
#archive = zipfile.ZipFile(path_2017c, 'r')
#imgdata = archive.read('ISIC_0000143.jpg')

# Example of how to define the file
#file = path_2017 + "\\" + "ISIC_0000143.jpg"

# Create anchor pictures:
# From folder (not Zip)
print(missing_anchors)
if (missing_anchors):
    print("Begin: copying and collecting anchor images ...")
    for i in images_2017:
        file = path_2017 + "\\" + i + ".jpg"
        shutil.copy(file, dest_2017)
    print("Done: copying and collecting anchor images")


False


In [10]:
# Create unlabeled pictures:
# From Zip-file

print(missing_unlabeled)
if(missing_unlabeled):
    archive = zipfile.ZipFile(path_2020, 'r')
    file = "train/" + images_2020[0] + ".dcm"
    #print(file)
    #archive.extract(file, dest_2020)

    print("Begin: copying and collecting anchor images ...")
    for i in images_2020:
        file = "train/" + i + ".dcm"
        archive.extract(file, dest_2020)
    print("Done: copying and collecting anchor images")
    

False
